# Dermatology AI Agents Lab

## Specialized AI Agents for Skincare and Clinical Dermatology

This lab implements AI agents specialized for dermatological consultation, integrating with the SkinTwin cognitive architecture.

### Available Agents

| Agent | Purpose | Target User | Safety Level |
|-------|---------|-------------|-------------|
| **Skincare Consultant** | Personalized skincare advice | Consumers | Consumer |
| **Clinical Assistant** | Clinical decision support | Healthcare Professionals | Professional |
| **Product Advisor** | Product recommendations | Consumers | Consumer |

### Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    RegimAI Gateway                          │
├─────────────────────────────────────────────────────────────┤
│  Skincare      │   Clinical      │   Product               │
│  Consultant    │   Assistant     │   Advisor               │
│  Agent         │   Agent         │   Agent                 │
├─────────────────────────────────────────────────────────────┤
│           SkinTwin Cognitive Architecture                   │
│  AtomSpace │ PLN │ MOSES │ ESN │ ECAN                      │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Environment Setup
import os
import json
import uuid
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum

from dotenv import load_dotenv
load_dotenv()

GATEWAY_URL = os.getenv('REGIMA_GATEWAY_URL', 'http://localhost:3000')
API_KEY = os.getenv('REGIMA_API_KEY', 'demo-key')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')

## 1. Agent Base Classes

In [ ]:
class AgentType(Enum):
    SKINCARE_CONSULTANT = "skincare_consultant"
    CLINICAL_ASSISTANT = "clinical_assistant"
    PRODUCT_ADVISOR = "product_advisor"

@dataclass
class Message:
    """A single message in a conversation"""
    role: str  # "user", "assistant", "system"
    content: str
    timestamp: datetime = field(default_factory=datetime.now)
    metadata: Dict = field(default_factory=dict)
    
    def to_dict(self) -> Dict:
        return {
            "role": self.role,
            "content": self.content,
            "timestamp": self.timestamp.isoformat(),
            "metadata": self.metadata
        }

@dataclass
class ConversationSession:
    """Manages a conversation session with an agent"""
    session_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    agent_type: AgentType = AgentType.SKINCARE_CONSULTANT
    messages: List[Message] = field(default_factory=list)
    context: Dict = field(default_factory=dict)
    created_at: datetime = field(default_factory=datetime.now)
    
    def add_message(self, role: str, content: str, metadata: Dict = None):
        self.messages.append(Message(
            role=role,
            content=content,
            metadata=metadata or {}
        ))
    
    def get_conversation_history(self) -> List[Dict]:
        return [m.to_dict() for m in self.messages]
    
    def to_dict(self) -> Dict:
        return {
            "session_id": self.session_id,
            "agent_type": self.agent_type.value,
            "messages": self.get_conversation_history(),
            "context": self.context,
            "created_at": self.created_at.isoformat()
        }

print("Base classes loaded successfully")

## 2. Skincare Consultant Agent

Provides personalized skincare advice for consumers.

In [ ]:
class SkincareConsultantAgent:
    """AI agent for personalized skincare consultation"""
    
    SYSTEM_PROMPT = """You are a knowledgeable skincare consultant AI assistant specialized in dermatology and skincare.

Your role is to:
- Provide personalized skincare routine recommendations
- Explain skincare ingredients and their benefits
- Help users understand their skin type and concerns
- Suggest appropriate products for different skin types
- Educate about sun protection and anti-aging

Important guidelines:
- Never diagnose medical conditions - recommend professional consultation for persistent issues
- Focus on over-the-counter skincare products only
- Always recommend sunscreen and sun protection
- Be clear about ingredient interactions and potential sensitivities
- Acknowledge limitations and when to see a dermatologist

Always include a disclaimer that your advice is informational and not a substitute for professional medical advice."""
    
    # Skin type knowledge base
    SKIN_TYPE_RECOMMENDATIONS = {
        "oily": {
            "cleanser": "gel or foaming cleanser with salicylic acid",
            "moisturizer": "lightweight, oil-free, gel-based moisturizer",
            "key_ingredients": ["niacinamide", "salicylic acid", "hyaluronic acid", "zinc"],
            "avoid": ["heavy oils", "thick creams", "coconut oil"],
            "tips": ["Don't over-cleanse", "Use blotting papers", "Don't skip moisturizer"]
        },
        "dry": {
            "cleanser": "cream or milk cleanser, no foaming agents",
            "moisturizer": "rich cream with ceramides and hyaluronic acid",
            "key_ingredients": ["ceramides", "hyaluronic acid", "squalane", "shea butter"],
            "avoid": ["alcohol-based products", "harsh sulfates", "fragranced products"],
            "tips": ["Apply products on damp skin", "Use humidifier", "Layer hydrating products"]
        },
        "combination": {
            "cleanser": "gentle gel cleanser",
            "moisturizer": "lightweight lotion, heavier on dry areas",
            "key_ingredients": ["niacinamide", "hyaluronic acid", "green tea extract"],
            "avoid": ["one-size-fits-all heavy products"],
            "tips": ["Multi-mask different zones", "Adjust routine seasonally"]
        },
        "sensitive": {
            "cleanser": "fragrance-free, gentle micellar water or cream cleanser",
            "moisturizer": "fragrance-free, minimal ingredient moisturizer",
            "key_ingredients": ["centella asiatica", "aloe vera", "oat extract", "allantoin"],
            "avoid": ["fragrance", "essential oils", "alcohol", "harsh actives"],
            "tips": ["Patch test everything", "Introduce one product at a time", "Keep routine simple"]
        },
        "normal": {
            "cleanser": "gentle cleanser of choice",
            "moisturizer": "balanced moisturizer",
            "key_ingredients": ["vitamin C", "peptides", "antioxidants"],
            "avoid": ["over-treating"],
            "tips": ["Focus on prevention", "Maintain balance"]
        }
    }
    
    CONCERN_TREATMENTS = {
        "acne": {
            "ingredients": ["benzoyl peroxide", "salicylic acid", "niacinamide", "retinoids"],
            "routine_tips": "Cleanse twice daily, use non-comedogenic products",
            "when_to_see_derm": "Severe, cystic, or scarring acne"
        },
        "aging": {
            "ingredients": ["retinol", "vitamin C", "peptides", "hyaluronic acid"],
            "routine_tips": "Consistent sunscreen use is the best anti-aging",
            "when_to_see_derm": "For prescription-strength retinoids"
        },
        "hyperpigmentation": {
            "ingredients": ["vitamin C", "niacinamide", "alpha arbutin", "azelaic acid"],
            "routine_tips": "Sun protection is crucial to prevent worsening",
            "when_to_see_derm": "Persistent dark spots, melasma"
        },
        "dryness": {
            "ingredients": ["hyaluronic acid", "ceramides", "glycerin", "squalane"],
            "routine_tips": "Apply moisturizer on damp skin, avoid hot water",
            "when_to_see_derm": "Persistent dryness with flaking or cracking"
        },
        "redness": {
            "ingredients": ["centella asiatica", "niacinamide", "green tea", "azelaic acid"],
            "routine_tips": "Avoid triggers like heat, alcohol, spicy food",
            "when_to_see_derm": "Persistent redness, visible blood vessels (may be rosacea)"
        }
    }
    
    def __init__(self):
        self.sessions: Dict[str, ConversationSession] = {}
    
    def create_session(self, context: Dict = None) -> ConversationSession:
        """Create a new consultation session"""
        session = ConversationSession(
            agent_type=AgentType.SKINCARE_CONSULTANT,
            context=context or {}
        )
        session.add_message("system", self.SYSTEM_PROMPT)
        self.sessions[session.session_id] = session
        return session
    
    def get_skin_type_advice(self, skin_type: str) -> Dict:
        """Get recommendations for a specific skin type"""
        skin_type = skin_type.lower()
        if skin_type not in self.SKIN_TYPE_RECOMMENDATIONS:
            return {"error": f"Unknown skin type: {skin_type}"}
        
        recs = self.SKIN_TYPE_RECOMMENDATIONS[skin_type]
        return {
            "skin_type": skin_type,
            "recommendations": recs,
            "basic_routine": self._generate_basic_routine(skin_type)
        }
    
    def get_concern_advice(self, concern: str) -> Dict:
        """Get treatment advice for a specific concern"""
        concern = concern.lower()
        if concern not in self.CONCERN_TREATMENTS:
            return {"error": f"No specific advice for concern: {concern}"}
        
        treatment = self.CONCERN_TREATMENTS[concern]
        return {
            "concern": concern,
            "treatment_info": treatment,
            "disclaimer": "This is general information. Consult a dermatologist for persistent concerns."
        }
    
    def _generate_basic_routine(self, skin_type: str) -> Dict:
        """Generate a basic skincare routine"""
        recs = self.SKIN_TYPE_RECOMMENDATIONS.get(skin_type, {})
        
        return {
            "morning": [
                {"step": 1, "product": "Cleanser", "recommendation": recs.get("cleanser", "gentle cleanser")},
                {"step": 2, "product": "Toner/Essence", "recommendation": "Optional, hydrating toner"},
                {"step": 3, "product": "Serum", "recommendation": "Vitamin C or antioxidant serum"},
                {"step": 4, "product": "Moisturizer", "recommendation": recs.get("moisturizer", "moisturizer")},
                {"step": 5, "product": "Sunscreen", "recommendation": "SPF 30+ broad spectrum (essential!)"}
            ],
            "evening": [
                {"step": 1, "product": "Cleanser", "recommendation": "Oil cleanser or micellar water (if wearing makeup)"},
                {"step": 2, "product": "Cleanser", "recommendation": recs.get("cleanser", "gentle cleanser")},
                {"step": 3, "product": "Treatment", "recommendation": "Active treatment (retinol, AHA/BHA, etc.)"},
                {"step": 4, "product": "Moisturizer", "recommendation": recs.get("moisturizer", "moisturizer")}
            ]
        }
    
    def analyze_routine(self, routine: List[Dict]) -> Dict:
        """Analyze a user's current routine"""
        analysis = {
            "products_count": len(routine),
            "has_cleanser": False,
            "has_moisturizer": False,
            "has_sunscreen": False,
            "potential_conflicts": [],
            "suggestions": []
        }
        
        ingredients_used = []
        for product in routine:
            product_type = product.get("type", "").lower()
            ingredients = product.get("ingredients", [])
            ingredients_used.extend(ingredients)
            
            if "cleanser" in product_type:
                analysis["has_cleanser"] = True
            if "moisturizer" in product_type or "cream" in product_type:
                analysis["has_moisturizer"] = True
            if "sunscreen" in product_type or "spf" in product_type:
                analysis["has_sunscreen"] = True
        
        # Check for conflicts
        conflict_pairs = [
            ("vitamin_c", "niacinamide"),  # Often fine but some people have issues
            ("retinol", "aha"),
            ("retinol", "bha"),
            ("retinol", "vitamin_c"),
            ("aha", "bha")  # When used together can be irritating
        ]
        
        for ing1, ing2 in conflict_pairs:
            if ing1 in ingredients_used and ing2 in ingredients_used:
                analysis["potential_conflicts"].append({
                    "ingredients": [ing1, ing2],
                    "advice": "Consider using on alternate days or different times of day"
                })
        
        # Generate suggestions
        if not analysis["has_sunscreen"]:
            analysis["suggestions"].append("Add a broad-spectrum SPF 30+ sunscreen to your morning routine")
        if not analysis["has_moisturizer"]:
            analysis["suggestions"].append("Add a moisturizer to help maintain skin barrier")
        if not analysis["has_cleanser"]:
            analysis["suggestions"].append("Add a gentle cleanser to remove impurities")
        
        return analysis

# Initialize agent
skincare_agent = SkincareConsultantAgent()
print("Skincare Consultant Agent loaded")

In [ ]:
# Demo: Get skin type advice
print("=" * 60)
print("SKINCARE CONSULTANT - Skin Type Advice")
print("=" * 60)

advice = skincare_agent.get_skin_type_advice("oily")
print(f"\nSkin Type: {advice['skin_type'].upper()}")
print(f"\nRecommended Cleanser: {advice['recommendations']['cleanser']}")
print(f"Recommended Moisturizer: {advice['recommendations']['moisturizer']}")
print(f"\nKey Ingredients: {', '.join(advice['recommendations']['key_ingredients'])}")
print(f"Avoid: {', '.join(advice['recommendations']['avoid'])}")
print(f"\nTips:")
for tip in advice['recommendations']['tips']:
    print(f"  • {tip}")

## 3. Clinical Dermatology Assistant Agent

Provides clinical decision support for healthcare professionals.

In [ ]:
class ClinicalDermatologyAssistant:
    """AI agent for clinical dermatology decision support"""
    
    SYSTEM_PROMPT = """You are a clinical dermatology assistant AI providing decision support for healthcare professionals.

Your role is to:
- Assist with differential diagnosis based on clinical presentations
- Provide evidence-based treatment recommendations
- Reference clinical guidelines and research
- Support clinical decision-making with relevant information
- Help with patient education materials

Important guidelines:
- All recommendations should cite evidence levels when available
- Emphasize that final clinical decisions rest with the treating physician
- Flag potential drug interactions and contraindications
- Note when referral to specialist may be indicated
- Maintain HIPAA compliance - never store identifiable patient information

This assistant is for licensed healthcare professionals only. All clinical decisions remain the responsibility of the treating provider."""
    
    # Clinical knowledge base
    DIFFERENTIAL_DIAGNOSES = {
        "papulopustular_face": {
            "differentials": [
                {"condition": "Acne Vulgaris", "likelihood": "high", "key_features": ["comedones", "inflammatory lesions", "sebaceous areas"]},
                {"condition": "Rosacea", "likelihood": "moderate", "key_features": ["central facial erythema", "flushing history", "no comedones"]},
                {"condition": "Perioral Dermatitis", "likelihood": "moderate", "key_features": ["perioral distribution", "sparing of vermilion border", "history of topical steroids"]},
                {"condition": "Folliculitis", "likelihood": "low", "key_features": ["follicular distribution", "may be bacterial or fungal"]}
            ]
        },
        "scaly_plaques": {
            "differentials": [
                {"condition": "Psoriasis", "likelihood": "high", "key_features": ["silvery scale", "Auspitz sign", "extensor surfaces", "nail changes"]},
                {"condition": "Eczema/Atopic Dermatitis", "likelihood": "moderate", "key_features": ["flexural distribution", "pruritus", "atopic history"]},
                {"condition": "Seborrheic Dermatitis", "likelihood": "moderate", "key_features": ["greasy scale", "nasolabial folds", "scalp involvement"]},
                {"condition": "Tinea Corporis", "likelihood": "low", "key_features": ["annular lesions", "central clearing", "active border"]}
            ]
        },
        "pigmented_lesion": {
            "differentials": [
                {"condition": "Benign Nevus", "likelihood": "high", "key_features": ["symmetric", "regular borders", "uniform color"]},
                {"condition": "Seborrheic Keratosis", "likelihood": "moderate", "key_features": ["stuck-on appearance", "waxy surface", "horn cysts"]},
                {"condition": "Melanoma", "likelihood": "low", "key_features": ["ABCDE criteria", "change over time", "irregular features"]},
                {"condition": "Lentigo", "likelihood": "moderate", "key_features": ["flat", "sun-exposed areas", "uniform color"]}
            ],
            "urgent_referral_criteria": ["Asymmetry", "Border irregularity", "Color variation", "Diameter >6mm", "Evolution/change"]
        }
    }
    
    TREATMENT_GUIDELINES = {
        "acne_vulgaris": {
            "mild": {
                "first_line": ["Topical retinoid", "Benzoyl peroxide"],
                "evidence_level": "A",
                "duration": "8-12 weeks for initial response"
            },
            "moderate": {
                "first_line": ["Topical retinoid + topical antibiotic", "Benzoyl peroxide + topical antibiotic"],
                "second_line": ["Oral antibiotics + topical retinoid"],
                "evidence_level": "A",
                "duration": "3 months, reassess"
            },
            "severe": {
                "first_line": ["Oral isotretinoin"],
                "second_line": ["Oral antibiotics + topical combination", "Hormonal therapy (females)"],
                "evidence_level": "A",
                "considerations": ["iPLEDGE program for isotretinoin", "Baseline labs required"]
            }
        },
        "psoriasis": {
            "mild": {
                "first_line": ["Topical corticosteroids", "Vitamin D analogues"],
                "evidence_level": "A",
                "duration": "2-4 weeks for initial response"
            },
            "moderate_to_severe": {
                "first_line": ["Phototherapy", "Methotrexate", "Biologics"],
                "evidence_level": "A",
                "considerations": ["Screen for TB before biologics", "Monitor LFTs with methotrexate"]
            }
        },
        "eczema": {
            "mild": {
                "first_line": ["Emollients", "Low-potency topical corticosteroids"],
                "evidence_level": "A"
            },
            "moderate_to_severe": {
                "first_line": ["Mid-to-high potency topical corticosteroids", "Topical calcineurin inhibitors"],
                "second_line": ["Dupilumab", "Phototherapy"],
                "evidence_level": "A"
            }
        }
    }
    
    def __init__(self, require_credentials: bool = True):
        self.require_credentials = require_credentials
        self.sessions: Dict[str, ConversationSession] = {}
    
    def verify_credentials(self, credentials: Dict) -> bool:
        """Verify healthcare professional credentials"""
        # In production, this would verify against a credential service
        required_fields = ["license_number", "license_type", "state"]
        return all(field in credentials for field in required_fields)
    
    def create_session(self, credentials: Dict, context: Dict = None) -> ConversationSession:
        """Create a new clinical session"""
        if self.require_credentials and not self.verify_credentials(credentials):
            raise ValueError("Valid healthcare professional credentials required")
        
        session = ConversationSession(
            agent_type=AgentType.CLINICAL_ASSISTANT,
            context={"credentials_verified": True, **(context or {})}
        )
        session.add_message("system", self.SYSTEM_PROMPT)
        self.sessions[session.session_id] = session
        return session
    
    def get_differential_diagnosis(self, presentation_type: str, clinical_features: List[str] = None) -> Dict:
        """Generate differential diagnosis based on presentation"""
        presentation_type = presentation_type.lower().replace(" ", "_")
        
        if presentation_type not in self.DIFFERENTIAL_DIAGNOSES:
            return {
                "error": "Presentation type not in database",
                "suggestion": "Please describe the clinical presentation in detail"
            }
        
        ddx = self.DIFFERENTIAL_DIAGNOSES[presentation_type]
        
        # Adjust likelihood based on provided clinical features
        differentials = ddx["differentials"].copy()
        if clinical_features:
            for diff in differentials:
                matches = sum(1 for f in clinical_features if f.lower() in [kf.lower() for kf in diff["key_features"]])
                if matches > 0:
                    diff["feature_matches"] = matches
        
        return {
            "presentation": presentation_type,
            "differential_diagnoses": differentials,
            "urgent_referral_criteria": ddx.get("urgent_referral_criteria", []),
            "disclaimer": "Clinical decision support only. Final diagnosis is the responsibility of the treating provider."
        }
    
    def get_treatment_recommendation(self, condition: str, severity: str) -> Dict:
        """Get evidence-based treatment recommendations"""
        condition = condition.lower().replace(" ", "_")
        severity = severity.lower()
        
        if condition not in self.TREATMENT_GUIDELINES:
            return {"error": f"No treatment guidelines for: {condition}"}
        
        guidelines = self.TREATMENT_GUIDELINES[condition]
        
        if severity not in guidelines:
            available = list(guidelines.keys())
            return {"error": f"Severity '{severity}' not found. Available: {available}"}
        
        treatment = guidelines[severity]
        
        return {
            "condition": condition,
            "severity": severity,
            "treatment_recommendations": treatment,
            "evidence_level": treatment.get("evidence_level", "N/A"),
            "disclaimer": "Treatment decisions should be individualized based on patient factors. This is decision support only."
        }

# Initialize agent (credentials not required for demo)
clinical_agent = ClinicalDermatologyAssistant(require_credentials=False)
print("Clinical Dermatology Assistant loaded")

In [ ]:
# Demo: Get differential diagnosis
print("=" * 60)
print("CLINICAL ASSISTANT - Differential Diagnosis")
print("=" * 60)

ddx = clinical_agent.get_differential_diagnosis(
    "papulopustular_face",
    clinical_features=["comedones", "inflammatory lesions"]
)

print(f"\nPresentation: {ddx['presentation']}")
print(f"\nDifferential Diagnoses:")
for diff in ddx['differential_diagnoses']:
    print(f"\n  {diff['condition']} (Likelihood: {diff['likelihood']})")
    print(f"    Key features: {', '.join(diff['key_features'])}")
    if 'feature_matches' in diff:
        print(f"    Feature matches: {diff['feature_matches']}")

In [ ]:
# Demo: Get treatment recommendation
print("\n" + "=" * 60)
print("CLINICAL ASSISTANT - Treatment Guidelines")
print("=" * 60)

treatment = clinical_agent.get_treatment_recommendation("acne_vulgaris", "moderate")

print(f"\nCondition: {treatment['condition']}")
print(f"Severity: {treatment['severity']}")
print(f"Evidence Level: {treatment['evidence_level']}")
print(f"\nFirst-line treatments:")
for tx in treatment['treatment_recommendations']['first_line']:
    print(f"  • {tx}")
if 'second_line' in treatment['treatment_recommendations']:
    print(f"\nSecond-line treatments:")
    for tx in treatment['treatment_recommendations']['second_line']:
        print(f"  • {tx}")

## 4. Product Advisor Agent

In [ ]:
class ProductAdvisorAgent:
    """AI agent for skincare product recommendations and ingredient analysis"""
    
    INGREDIENT_DATABASE = {
        "retinol": {
            "category": "retinoid",
            "benefits": ["anti-aging", "acne", "cell turnover"],
            "concerns": ["irritation", "sun sensitivity"],
            "avoid_with": ["vitamin_c", "aha", "bha"],
            "best_for": ["aging", "acne", "texture"]
        },
        "niacinamide": {
            "category": "vitamin",
            "benefits": ["pore minimizing", "brightening", "barrier support"],
            "concerns": ["flushing at high concentrations"],
            "avoid_with": [],
            "best_for": ["oily skin", "acne", "hyperpigmentation"]
        },
        "hyaluronic_acid": {
            "category": "humectant",
            "benefits": ["hydration", "plumping"],
            "concerns": ["can dry out skin in low humidity"],
            "avoid_with": [],
            "best_for": ["all skin types", "dehydration"]
        },
        "salicylic_acid": {
            "category": "bha",
            "benefits": ["exfoliation", "pore clearing", "anti-inflammatory"],
            "concerns": ["drying", "sun sensitivity"],
            "avoid_with": ["retinol", "aha"],
            "best_for": ["oily skin", "acne", "blackheads"]
        },
        "vitamin_c": {
            "category": "antioxidant",
            "benefits": ["brightening", "antioxidant", "collagen synthesis"],
            "concerns": ["oxidation", "irritation"],
            "avoid_with": ["retinol"],
            "best_for": ["hyperpigmentation", "dullness", "aging"]
        },
        "glycolic_acid": {
            "category": "aha",
            "benefits": ["exfoliation", "brightening", "texture improvement"],
            "concerns": ["irritation", "sun sensitivity"],
            "avoid_with": ["retinol", "bha"],
            "best_for": ["dull skin", "texture", "hyperpigmentation"]
        },
        "ceramides": {
            "category": "lipid",
            "benefits": ["barrier repair", "moisture retention"],
            "concerns": [],
            "avoid_with": [],
            "best_for": ["dry skin", "eczema", "compromised barrier"]
        }
    }
    
    def __init__(self):
        self.sessions: Dict[str, ConversationSession] = {}
    
    def analyze_ingredients(self, ingredients: List[str]) -> Dict:
        """Analyze a list of product ingredients"""
        analysis = {
            "ingredients_found": [],
            "potential_conflicts": [],
            "combined_benefits": set(),
            "warnings": []
        }
        
        normalized = [i.lower().replace(" ", "_") for i in ingredients]
        
        for ing in normalized:
            if ing in self.INGREDIENT_DATABASE:
                info = self.INGREDIENT_DATABASE[ing]
                analysis["ingredients_found"].append({
                    "name": ing,
                    "category": info["category"],
                    "benefits": info["benefits"]
                })
                analysis["combined_benefits"].update(info["benefits"])
                
                # Check for conflicts
                for avoid in info["avoid_with"]:
                    if avoid in normalized:
                        analysis["potential_conflicts"].append({
                            "ingredients": [ing, avoid],
                            "recommendation": "Use at different times (AM/PM) or alternate days"
                        })
        
        analysis["combined_benefits"] = list(analysis["combined_benefits"])
        
        return analysis
    
    def recommend_for_concern(self, concern: str, skin_type: str = "normal") -> Dict:
        """Recommend ingredients based on skin concern"""
        concern = concern.lower()
        
        recommendations = []
        for name, info in self.INGREDIENT_DATABASE.items():
            if concern in info["best_for"] or any(concern in b for b in info["best_for"]):
                recommendations.append({
                    "ingredient": name.replace("_", " ").title(),
                    "benefits": info["benefits"],
                    "concerns": info["concerns"]
                })
        
        return {
            "concern": concern,
            "skin_type": skin_type,
            "recommended_ingredients": recommendations,
            "note": "Introduce new actives gradually and always patch test first"
        }

# Initialize agent
product_agent = ProductAdvisorAgent()
print("Product Advisor Agent loaded")

In [ ]:
# Demo: Analyze ingredients
print("=" * 60)
print("PRODUCT ADVISOR - Ingredient Analysis")
print("=" * 60)

analysis = product_agent.analyze_ingredients(["retinol", "niacinamide", "vitamin C"])

print(f"\nIngredients analyzed: {len(analysis['ingredients_found'])}")
print(f"\nCombined benefits: {', '.join(analysis['combined_benefits'])}")

if analysis['potential_conflicts']:
    print(f"\n⚠️  Potential conflicts:")
    for conflict in analysis['potential_conflicts']:
        print(f"  • {conflict['ingredients'][0]} + {conflict['ingredients'][1]}")
        print(f"    Recommendation: {conflict['recommendation']}")

## 5. Gateway Integration

In [ ]:
import requests

class DermatologyAgentsClient:
    """Client for dermatology agents through RegimAI Gateway"""
    
    def __init__(self, gateway_url: str, api_key: str):
        self.gateway_url = gateway_url.rstrip('/')
        self.api_key = api_key
        self.headers = {
            "X-API-Key": api_key,
            "Content-Type": "application/json"
        }
    
    def chat_skincare_consultant(self, message: str, session_id: str = None, context: Dict = None) -> Dict:
        """Chat with skincare consultant agent"""
        payload = {
            "message": message,
            "session_id": session_id,
            "context": context or {}
        }
        
        response = requests.post(
            f"{self.gateway_url}/agents/skincare-consultant/chat",
            headers=self.headers,
            json=payload
        )
        return response.json()
    
    def chat_clinical_assistant(
        self,
        message: str,
        session_id: str = None,
        clinical_context: Dict = None,
        credentials: str = None
    ) -> Dict:
        """Chat with clinical assistant agent"""
        headers = {**self.headers}
        if credentials:
            headers["X-Professional-Credentials"] = credentials
        
        payload = {
            "message": message,
            "session_id": session_id,
            "clinical_context": clinical_context or {}
        }
        
        response = requests.post(
            f"{self.gateway_url}/agents/clinical-assistant/chat",
            headers=headers,
            json=payload
        )
        return response.json()
    
    def get_product_recommendations(self, skin_type: str, concerns: List[str]) -> Dict:
        """Get product recommendations"""
        payload = {
            "skin_type": skin_type,
            "concerns": concerns
        }
        
        response = requests.post(
            f"{self.gateway_url}/agents/product-advisor/recommend",
            headers=self.headers,
            json=payload
        )
        return response.json()

print("Gateway client ready")
print(f"Gateway URL: {GATEWAY_URL}")

## Summary

This lab implements three specialized AI agents:

1. **Skincare Consultant**: Consumer-focused skincare advice
2. **Clinical Assistant**: Professional clinical decision support
3. **Product Advisor**: Ingredient analysis and product recommendations

### Integration Points

- **AtomSpace**: Store ingredient relationships and treatment knowledge
- **PLN**: Reason about treatment efficacy and contraindications
- **MOSES**: Optimize treatment recommendations
- **ESN**: Predict treatment response trajectories